## Introduction to deep learning - exercise

## Example

*Human Activity Recognition using Smartphones* dataset

Dataset description:

*The experiments have been carried out with a group of 30 volunteers. Each person performed six activities
(WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING) wearing a smartphone.
Using its embedded accelerometer and gyroscope, we captured 3-axial linear acceleration and 3-axial angular velocity.
The experiments have been video-recorded to label the data manually.*

**Variables:**
For each record in the dataset it is provided:
* A 561-feature vector with time and frequency domain variables.
* Its activity label.
* An identifier of the subject who carried out the experiment.

More details at: https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones

### Loading and preparing the data

In [1]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
folder = ''  ## put here folder where the file HAR_clean.csv is located

Load the dataset: "HAR_clean.csv"

In [3]:
all_data = pd.read_csv(os.path.join(folder, 'HAR_clean.csv'), index_col=0)

Divide into input and output data

In [4]:
input_data = all_data.iloc[:,:-2].values.astype(np.float32)
input_data.shape

(10299, 561)

In [5]:
output_data = all_data.iloc[:,-1].values
output_data.shape

(10299,)

Divide the data into train and test, keeping 30% for the test

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(input_data, output_data, test_size=0.3)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

(7209, 561) (7209,)
(3090, 561) (3090,)


### Best shallow ML model

In [7]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':['linear', 'rbf'], 'C':[1, 10, 100,1000], 'gamma':[0.01, 0.001]}

svm_model_d = svm.SVC()
opt_model_d = GridSearchCV(svm_model_d, parameters)

opt_model_d.fit(X_train, y_train)
print (opt_model_d.best_estimator_)

SVC(C=1000, gamma=0.001)


In [8]:
opt_model_d.score(X_test, y_test)

0.983495145631068

### Training a DL model

In [9]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(y_train)

y_train_encoded = le.transform(y_train)
y_test_encoded = le.transform(y_test)

In [10]:
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train_encoded)

X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test_encoded)

In [ ]:
# DNN model - define here the model - complete

class HARNet(nn.Module):
    def __init__(self, in_features, n_classes):
        super().__init__()

        self.net = None ## complete here

    def forward(self, x):
        return self.net(x)


n_features = X_train.shape[1]
n_classes = len(np.unique(y_train))

model = HARNet(n_features, n_classes)

In [ ]:
# Loss & optimizer - complete !
criterion = None # complete here
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
from sklearn.metrics import accuracy_score

# Training loop
epochs = 30
batch_size = 128
train_losses = []
test_accuracies = []

for epoch in range(epochs):

    # ---- mini-batch training ----
    perm = torch.randperm(X_train.size(0))
    epoch_loss = 0

    for i in range(0, X_train.size(0), batch_size):
        idx = perm[i:i + batch_size]
        xb, yb = X_train[idx], y_train[idx]

        preds = model(xb)
        loss = criterion(preds, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    train_losses.append(epoch_loss)

    # ---- evaluation ----
    with torch.no_grad():
        logits = model(X_test)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy_score(y_test.numpy(), preds.numpy())
        test_accuracies.append(acc)

    print(f"Epoch {epoch+1:02d} | Loss: {epoch_loss:.3f} | Test Acc: {acc:.4f}")

In [ ]:
# Test set evaluation
print("\nFinal test accuracy:", test_accuracies[-1])